# Question: Writing Viterbi Algorithm for the Primer
Write the Viterbi algorithm to implment Nature Primer.

Here are some suggestions:

a. You can begin by first defining all the parameters, such as states, transition matrix, and emmision matrix etc.

b. You can write a function to exactly calculate the values mentioned in the primer, for example, you can define a function get_log_prob_of_a_given_path ("EEEEEEEEEEEEEEEEEE5IIIIIII", "CTTCATGTGAAAGCAGACGTAAGTCA"). This should output -41.22.

By doing the above two, you earn 1 mark.

Now, you have to implement this in real to get max likely path that would emmit the observed sequence. You MUST note that maximum likely path will just be Es, but that is okay. Implementation is the key.

In [1]:
# Imports
import numpy as np
import math

In [5]:
# Define states and matrices
states = ['E', '5', 'I']

# Transition probabilities (log-transformed)
trans_probs = {
    'E': {'E': math.log(0.9), '5': math.log(0.1), 'I': float('-inf')},
    '5': {'E': float('-inf'), '5': float('-inf'), 'I': math.log(1.0)},
    'I': {'E': float('-inf'), '5': float('-inf'), 'I': math.log(0.9)}
}

# Emission probabilities (log-transformed)
emit_probs = {
    'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T': math.log(0.25)},
    '5': {'A': math.log(0.05), 'C': float('-inf'), 'G': math.log(0.95), 'T': float('-inf')},
    'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)}
}

# Input DNA sequence
dna_seq = "AGGCTCTCCATAAGG"
seq_length = len(dna_seq)

# Initialize tables for Viterbi algorithm
viterbi = [{} for _ in range(seq_length)]
backtrack = [{} for _ in range(seq_length)]

# Initialize first position
for state in states:
    viterbi[0][state] = emit_probs[state][dna_seq[0]] if state == 'E' else float('-inf')

# Fill Viterbi and backtrack tables
for pos in range(1, seq_length):
    for curr_state in states:
        max_log_prob = float('-inf')
        prev_best_state = None
        for prev_state in states:
            log_prob = (viterbi[pos-1][prev_state] + 
                       trans_probs[prev_state][curr_state] + 
                       emit_probs[curr_state][dna_seq[pos]])
            if log_prob > max_log_prob:
                max_log_prob = log_prob
                prev_best_state = prev_state
        viterbi[pos][curr_state] = max_log_prob
        backtrack[pos][curr_state] = prev_best_state

# Backtrack to find the most likely state sequence
best_final_prob = float('-inf')
best_final_state = None
for state in states:
    if viterbi[seq_length-1][state] > best_final_prob:
        best_final_prob = viterbi[seq_length-1][state]
        best_final_state = state

# Reconstruct the state path
state_path = [best_final_state]
current_state = best_final_state
for pos in range(seq_length-1, 0, -1):
    current_state = backtrack[pos][current_state]
    state_path.insert(0, current_state)

print("maximum likely path:")
# Output the state path
print(''.join(state_path))


maximum likely path:
EEEEEEEEEEEEEEE


In [6]:
# Function to compute log likelihood for a given sequence and state path
def compute_sequence_log_likelihood(sequence, states):
    """Computes the log likelihood of a DNA sequence given a state path"""
    if len(sequence) != len(states):
        raise ValueError("Sequence and state path must have identical lengths")

    log_likelihood = 0.0
    for i in range(len(sequence)):
        if i == 0:
            if states[i] != 'E':
                return float('-inf')
            log_likelihood = emit_probs[states[i]][sequence[i]]
        else:
            log_likelihood += (trans_probs[states[i-1]][states[i]] + 
                              emit_probs[states[i]][sequence[i]])

    # Include terminal transition probability
    log_likelihood += math.log(0.1)
    
    return log_likelihood

# Test sequence and state path
test_seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
test_states = "EEEEEEEEEEEEEEEEEE5IIIIIII"

# Compute and print log likelihood
likelihood = compute_sequence_log_likelihood(test_seq, test_states)
print(f"Log probability of sequence given state path: {likelihood:.2f}")

Log probability of sequence given state path: -41.22
